In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from skimage.io import imread, imshow
import cv2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from sklearn import model_selection

In [2]:
# Extract features from the image
imagePath  = "/Users/anjalisingh/Downloads/memes_final_27.7/finalmemes_bw"
images = os.listdir(imagePath)
images.remove('.DS_Store')
#images

In [3]:
def extract_features(images):
    features = {}
    for i in images:
        image = imread(imagePath+"/"+i, as_gray=True) 
        s=(300,300)
        image = cv2.resize(image,s)
        #print(image.shape)
        features[i] = np.reshape(image, (300*300))
    return features
features = extract_features(images)

In [4]:
memes = pd.read_csv("/Users/anjalisingh/Downloads/memes_final_27.7/captions_reduced_8000.csv")
memes.head()

,s.no.,img_id,caption,top,bottom
0,1,/Y-U-No,meme generator users-y u no give me more upvotes?,meme generator users,y u no give me more upvotes?
1,2,/Y-U-No,steve jobs-y u no respawn?!,steve jobs,y u no respawn?!
2,3,/Y-U-No,commercial-y u no same volume as show!?,commercial,y u no same volume as show!?
3,4,/Y-U-No,Victoria-y u no tell us your secret?!,Victoria,y u no tell us your secret?!
4,5,/Y-U-No,KONY-Y u no take justin bieber,KONY,Y u no take justin bieber


In [5]:
memes.tail()

,s.no.,img_id,caption,top,bottom
8310,8312,/Lol-Face,and i will try-to fix you,and i will try,to fix you
8311,8313,/Lol-Face,Cannot wait to get my nexus 7 in the post-i bo...,Cannot wait to get my nexus 7 in the post,i bought a sd card to expand the memory
8312,8314,/Lol-Face,elder scrolls online-wooooooooooooooo,elder scrolls online,wooooooooooooooo
8313,8315,/Lol-Face,lololololololololololololololololololololololo...,lololololololololololololololololololololololol,it happens! :(
8314,8316,/Lol-Face,Amee-No Longer in TEAM,Amee,No Longer in TEAM


In [6]:
print(len(memes))
# test 20%---> 1663 
m=6652

8315


In [7]:
memes = memes.dropna(axis=0)

In [8]:
print(len(memes))

8315


In [9]:
memes['Key'] = "ABC"
for i in range(0,len(memes)):
    memes['Key'][i] = "meme"+str(i+1)+'.jpg'

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
memes

,s.no.,img_id,caption,top,bottom,Key
0,1,/Y-U-No,meme generator users-y u no give me more upvotes?,meme generator users,y u no give me more upvotes?,meme1.jpg
1,2,/Y-U-No,steve jobs-y u no respawn?!,steve jobs,y u no respawn?!,meme2.jpg
2,3,/Y-U-No,commercial-y u no same volume as show!?,commercial,y u no same volume as show!?,meme3.jpg
3,4,/Y-U-No,Victoria-y u no tell us your secret?!,Victoria,y u no tell us your secret?!,meme4.jpg
4,5,/Y-U-No,KONY-Y u no take justin bieber,KONY,Y u no take justin bieber,meme5.jpg
...,...,...,...,...,...,...
8310,8312,/Lol-Face,and i will try-to fix you,and i will try,to fix you,meme8311.jpg
8311,8313,/Lol-Face,Cannot wait to get my nexus 7 in the post-i bo...,Cannot wait to get my nexus 7 in the post,i bought a sd card to expand the memory,meme8312.jpg
8312,8314,/Lol-Face,elder scrolls online-wooooooooooooooo,elder scrolls online,wooooooooooooooo,meme8313.jpg
8313,8315,/Lol-Face,lololololololololololololololololololololololo...,lololololololololololololololololololololololol,it happens! :(,meme8314.jpg


In [11]:
from sklearn.utils import shuffle
memes = shuffle(memes)

In [43]:
def create_tokenizer(descriptions):
    desc=list()
    for i in range(m):
        desc.append(memes['caption'][i])
    tokenizer = Tokenizer()
    # fit_on_texts: Updates internal vocabulary based on a list of texts. This method creates the vocabulary 
    # index based on word frequency
    tokenizer.fit_on_texts(desc)
    return tokenizer

tokenizer = create_tokenizer(memes['caption'])
vocab_size = len(tokenizer.word_index) + 1
vocab_size

9652

In [50]:
word = word_for_id(9651, tokenizer)

In [51]:
word

'willpower'

In [107]:
def max_length(descriptions):
    desc=list()
    for i in range(m):
        desc.append(memes['caption'][i])
    return max(len(d.split()) for d in desc)

max_length = max_length(memes['caption'])
max_length

47

In [108]:
# X1, X2, y --> X1: features
X1, X2, y = list(), list(), list()
for i in range(m):
    feat = features["meme"+str(i+1)+".jpg"][0]
    seq = tokenizer.texts_to_sequences(memes['caption'][i])[0]
    #print(len(seq))
    for s in range(0, len(seq)):
        in_seq, out_seq = seq[:s], seq[s]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        X1.append(feat)
        X2.append(in_seq)
        y.append(out_seq)

X2 = np.array(X2)
X1 = np.array(X1)
y = np.array(y)

In [109]:
X2.shape, X1.shape, y.shape

((6520, 47), (6520,), (6520, 9652))

In [110]:
# features from the CNN model squeezed from 2048 to 256 nodes
inputs1 = Input(shape=(300,300,1))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
    
# LSTM sequence Model
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
    
# Merging both models
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
# tie it together [image, seq][word]
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
# model.layers[2].set_weights([emb_matrix]) 
# model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [111]:
model.fit([X1, X2], y, epochs = 100, batch_size = 256) 
    
# summarize model
print(model.summary())

Epoch 1/100
26/26 [==============================] - 13s 348ms/step - loss: 9.0412
Epoch 2/100
26/26 [==============================] - 9s 359ms/step - loss: 6.4821
Epoch 3/100
26/26 [==============================] - 9s 363ms/step - loss: 3.1562
Epoch 4/100
26/26 [==============================] - 9s 361ms/step - loss: 3.0482
Epoch 5/100
26/26 [==============================] - 11s 407ms/step - loss: 3.0159
Epoch 6/100
26/26 [==============================] - 10s 403ms/step - loss: 2.9898
Epoch 7/100
26/26 [==============================] - 11s 432ms/step - loss: 2.9955
Epoch 8/100
26/26 [==============================] - 17s 647ms/step - loss: 2.9745
Epoch 9/100
26/26 [==============================] - 10s 381ms/step - loss: 2.9557
Epoch 10/100
26/26 [==============================] - 11s 424ms/step - loss: 2.9501
Epoch 11/100
26/26 [==============================] - 11s 406ms/step - loss: 2.9667
Epoch 12/100
26/26 [==============================] - 11s 419ms/step - loss: 2.9453
Epoc

### Testing

In [15]:
def create_tokenizer(descriptions):
    desc=list()
    for i in range(m+1, len(memes)):
        desc.append(memes['caption'][i])
    tokenizer = Tokenizer()
    # fit_on_texts: Updates internal vocabulary based on a list of texts. This method creates the vocabulary 
    # index based on word frequency
    tokenizer.fit_on_texts(desc)
    return tokenizer

tokenizer = create_tokenizer(memes['caption'])
vocab_size = len(tokenizer.word_index) + 1
vocab_size

4448

In [16]:
def max_length(descriptions):
    desc=list()
    for i in range(m+1, len(memes)):
        desc.append(memes['caption'][i])
    return max(len(d.split()) for d in desc)

max_length = max_length(memes['caption'])
max_length

46

In [17]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [18]:
def extract_features(images):
    image = imread(images, as_gray=True) 
    s=(300,300)
    image = cv2.resize(image,s)
        #print(image.shape)
    features = np.reshape(image, (300*300))
    return features

In [19]:
model = load_model('modelTR.h5')

In [28]:
# img_path="/Users/anjalisingh/Downloads/memes_final_27.7/finalmemes_bw/meme8313.jpg" --> 178967385
img_path="/Users/anjalisingh/Downloads/memes_final_27.7/finalmemes_bw/meme8313.jpg" # --> 178967385
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=47)
        print(sequence.shape, photo.shape)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        print(pred)
        word = word_for_id(pred, tokenizer)
        print(word)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

photo = extract_features(img_path)
photo.resize(1, 300, 300, 1)

In [29]:
t = generate_desc(model, tokenizer, photo,max_length)

(1, 47) (1, 300, 300, 1)
178967385
None


In [88]:
t

'start'